In [4]:
# imports

import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import torch as t
import torch.nn as nn
import torch.nn.functional as F

In [45]:
def generate_spiral_data(
    N: int = 100, # number of points per class
    K: int = 2, # number of classes
    R: int = 10, # radius limit
    S: int = 0.2 # noise standard deviation
):
    """
    Generate a spiral dataset
    """
    spirals = []
    for k in range(K):
        r = np.linspace(0.0, R, N)
        t = np.linspace(k * 2 * np.pi / K, (k + 4) * 2 * np.pi / K, N) + np.random.randn(N) * S
        x = r * np.sin(t)
        y = r * np.cos(t)
        spirals.append(np.vstack((x, y)).T)
    return np.stack(spirals)

In [50]:
data = generate_spiral_data(500, 2, 10)

In [51]:
data.shape

(2, 500, 2)

In [52]:
def plot_spiral(data, R):
    fig = go.Figure()
    fig = px.scatter(x=data[0, :, 0], y=data[0, :, 1], color_discrete_sequence=['red'])
    fig.add_scatter(x=data[1, :, 0], y=data[1, :, 1], mode='markers', marker_color='blue')
    fig.update_layout(
        xaxis=dict(
            title='x',
            showgrid=True,
            gridcolor='lightgray',
            # dtick=1,
            showline=True,
            linecolor='grey',
            zeroline=True,
            zerolinewidth=0,
            zerolinecolor='grey',
            linewidth=1,
            range=[-R, R],
            tickangle=-0,
        ),
        
        yaxis=dict(
            title='y',
            showgrid=True,
            gridcolor='lightgray',
            showline=True,
            zeroline=True,
            zerolinewidth=0,
            zerolinecolor='black',
            linewidth=1,
            linecolor='grey',
            tick0=0,
            range=[-R, R],
        ),
        )
    # Set background color to white
    fig.update_layout(plot_bgcolor='white')
    fig.update_traces(line={'width': 3.8})
    # make fig square
    fig.update_layout(
        width=500,
        height=500,
    )
    return fig

In [53]:
fig = plot_spiral(data, 10)
fig.show()

In [59]:
# data is shape (2, 500, 2), change it to (x, y, class) where class is 0 or 1
data = data.reshape(-1, 2)
data = np.hstack((data, np.zeros((data.shape[0], 1))))
data[500:, 2] = 1
data.shape

(1000, 3)

In [63]:
# shuffle data
np.random.shuffle(data)

In [64]:
data.shape

(1000, 3)

In [65]:
x = data[:, :2]
y = data[:, 2]

x_train = x[:800]
y_train = y[:800]
x_test = x[800:]
y_test = y[800:]

In [66]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((800, 2), (800,), (200, 2), (200,))

In [68]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import plotly.express as px

# Assuming you have your data loaded in variables x_train, y_train, x_test, y_test

# Convert NumPy arrays to PyTorch tensors
x_train, y_train, x_test, y_test = map(torch.tensor, (x_train, y_train, x_test, y_test))

# Create DataLoader for training and testing
train_dataset = TensorDataset(x_train.float(), y_train.long())
test_dataset = TensorDataset(x_test.float(), y_test.long())
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define an improved neural network
class ImprovedNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(ImprovedNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# Set hyperparameters
input_size = 2
hidden_size1 = 128
hidden_size2 = 64
output_size = 2
learning_rate = 0.001
num_epochs = 50

# Initialize model, loss, and optimizer
model = ImprovedNN(input_size, hidden_size1, hidden_size2, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
train_losses = []
for epoch in tqdm(range(num_epochs), desc="Training", unit="epoch"):
    total_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    train_losses.append(total_loss / len(train_loader))

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Testing", unit="batch"):
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Accuracy on test set: {accuracy * 100:.2f}%")

# Plot loss curve
fig = px.line(x=range(1, num_epochs + 1), y=train_losses, title='Training Loss Curve')
fig.update_layout(xaxis_title='Epoch', yaxis_title='Loss')
fig.show()

/tmp/ipykernel_2179519/1021938594.py:11: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

Testing: 100%|██████████| 7/7 [00:00<00:00, 2943.37batch/s]

Accuracy on test set: 92.50%


In [96]:
import numpy as np
import plotly.graph_objects as go

# Create a 10x10 grid
x_grid, y_grid = np.meshgrid(np.linspace(-15, 15, 10), np.linspace(-15, 15, 10))
grid_points = np.column_stack((x_grid.ravel(), y_grid.ravel()))

# Convert grid points to PyTorch tensor
grid_points_tensor = torch.tensor(grid_points, dtype=torch.float32)

# Evaluate the model on the grid points
model.eval()
with torch.no_grad():
    predictions = model(grid_points_tensor)

# Get predicted class labels
predicted_labels = torch.argmax(predictions, dim=1).numpy()

# Plot the decision boundary
fig = go.Figure()

# Scatter plot for spiral data
fig.add_trace(go.Scatter(
    x=x_train[:, 0], y=x_train[:, 1],
    mode='markers',
    marker=dict(color=y_train.numpy(), colorscale='thermal', size=8),
    showlegend=False,
    opacity=0.7,
))

# Contour plot for decision boundary
fig.add_trace(go.Contour(
    x=x_grid[0], y=y_grid[:, 0],
    z=predicted_labels.reshape(x_grid.shape),
    colorscale=[[0.2, 'rgb(200,200,255)'], [0.8, 'rgb(255,200,200)']],
    contours=dict(start=0.5, end=0.5, size=1),
    showscale=False,
    opacity=0.3,
))

fig.update_layout(title='Decision Boundary of the Neural Network',
                  xaxis_title='X-axis',
                  yaxis_title='Y-axis',
                  showlegend=False)

fig.update_layout(
    xaxis=dict(
        title='x',
        showgrid=True,
        gridcolor='lightgray',
        # dtick=1,
        showline=True,
        linecolor='grey',
        zeroline=True,
        zerolinewidth=0,
        zerolinecolor='grey',
        linewidth=1,
        tickangle=-0,
    ),
    
    yaxis=dict(
        title='y',
        showgrid=True,
        gridcolor='lightgray',
        showline=True,
        zeroline=True,
        zerolinewidth=0,
        zerolinecolor='black',
        linewidth=1,
        linecolor='grey',
        tick0=0,
    ),
    )
# Set background color to white
fig.update_layout(plot_bgcolor='white')
fig.update_traces(line={'width': 3.8})
# make fig square
fig.update_layout(
    width=500,
    height=500,
)

fig.show()